## CSE422 lab: Genetic Algorithm

#### Genetic Algorithm Pseudo code:



**function** GENETIC-ALGORITHM( population, FITNESS-FN) **returns** an individual 
 
> **inputs:** population- a set of individuals/chromosomes; FITNESS-FN- a function that measures the fitness of an individual

>**repeat** 
new_population $\leftarrow$ empty set 
>>**for** $i=1$ **to** size ($ population$) **do**
$$
\begin{array}{l}
x \leftarrow \text { RANDOM-SELECTION }(\text { population, FITNESS-FN }) \\
y \leftarrow \text { RANDOM-SELECTION }(\text { population, FITNESS-FN }) \\
child  \leftarrow \operatorname{CROSSOVER}(x, y)
\end{array}
$$
>>>**if** (some_random_number < mutation_threshold) **then** child$\leftarrow$ MUTATE ( child ) 

>>>add child to new_population 

>>population $\leftarrow$ new_population 

>**until** some individual is fit enough, or enough time has elapsed

>**return** the best individual in population, according to FITNESS-FN
 


###Importing Libraries

In [9]:
import numpy as np
import random
global min_fit_ind
global max_fitness_found
min_fit_ind = 0

### Fitness function

In [10]:
def fitness(population, n):

  '''calculates the fitness score of each
     of the individuals in the population
     
     returns a 1D numpy array: index referring to 
     ith individual in population, and value referring 
     to the fitness score.'''
  non_attacking_pair = {}
  for i in range(0,n):
    row = population[i]
    ans = 0
    #
    for j in range(0,n-1):
      for k in range(j+1,n):
        if row[k]==row[j]:
          ans += 1
      X = j
      Y = row[j]
      count = 1
      y= Y
      #Diagonally attacking pairs
      while (y+count)<n and X+count<n:
        if row[X+count] == y+count:
          ans += 1
        count += 1
      count = 1
      y= Y
      while (y-count)>=0 and X+count<n:
        if row[X+count] == y-count:
          ans += 1
        count += 1
    # Horizontally, Vertically and Diagonally non-attacking pairs 
    non_attacking_pair[i]=int(n*(n-1)/2 - ans)  
  fit = {}
  for i in range(0,n):
    fit[i]= non_attacking_pair[i]
  return fit

### Random Selection function

This built-in function might help to create the weighted random selection:

`numpy.random.choice(a, size, replace, p)` 

`p` are the weights of the individuals- value between 0 and 1; refers to the probability of each individual being selected.

`a` is the array

`size` how many samples to return

`replace = True`

In [11]:
def select(population, fit):
  ''' take input:  population and fit
      fit contains fitness values of each of the individuals 
      in the population  
      
      return:  one individual randomly giving
      more weight to ones which have high fitness score'''
  r = random.randint(0,n-2)  
  while r==min_fit_ind or (r+1)==min_fit_ind or visited[r]==1:
    r = random.randint(0,n-2)  
  visited[r]=1
  return r

### Crossover function


**function** CROSSOVER $(x, y)$ **returns** an individual 

>**inputs**: $x, y$  which are the parent individuals

>$n \leftarrow \mathrm{LENGTH}(x) ; c \leftarrow$ random number from 1 to $n$ 

>**return** APPEND(SUBSTRING $(x, 1, c),$ SUBSTRING $(y, c+1, n))$

In [12]:
def crossover(x, y):
  '''take input: 2 parents - x, y. 
     Generate a random crossover point. 
     Append first half of x with second 
     half of y to create the child
     
     returns: a child chromosome'''
  children = {}
  l = len(x)
  r = random.randint(0, l)
  child1 = np.concatenate((x[0:r],y[r:l]),axis=0)
  child2 = np.concatenate((y[0:r],x[r:l]),axis=0)
  children[0] = child1
  children[1] = child2   
  return children

###Mutation function

In [13]:
def mutate(child):
  '''take input: a child
     mutates a random 
     gene into another random gene
     
     returns: mutated child'''
  l = len(child)
  r_ind = random.randint(0,l-1)
  r_val = random.randint(0,l-1)
  child[r_ind] = r_val
  return child

###Print Chromosome and Fitness

In [14]:
def printAll(population, fit):
  for i in range(0,len(fit)):
    print("Chromosome:"+str(population[i])+"  "+"Fitness:"+str(fit[i]))
    m = int(len(fit)*(len(fit)-1)/2)
    if fit[i]>=m:
      global max_fitness_found
      max_fitness_found = True
      break
  print()     

### Genetic Algorithm Function

In [15]:
def GA(population, n, mutation_threshold = 0.3):
  '''implement the pseudocode here by
     calling the necessary functions- Fitness, 
     Selection, Crossover and Mutation
     
     print: the max fitness value and the 
     chromosome that generated it which is ultimately 
     the solution board'''
  fit = fitness(population, n)
  min_fit = fit[0] 
  min_fit_ind = 0
  for i in range(0,n):
    if fit[i]<min_fit:
      min_fit_ind = i 
      min_fit = fit[i]  
  printAll(population, fit)
  new_population = {} 
  count = 0    
  for i in range(0,int(n/2)):
    ind = select(population, fit)
    children = crossover(population[ind], population[ind+1])
    new_population[count]=children[0] 
    new_population[count+1]=children[1] 
    r = random.random()
    if r*0.01 < mutation_threshold:
      new_population[count] = mutate(new_population[count])
    r = random.random()
    if r*0.01 < mutation_threshold:
      new_population[count+1] = mutate(new_population[count+1])  
    count += 2
  return new_population

###Running the Genetic Algorithm function

In [16]:
'''for 8 queen problem, n = 8'''
n = 8

'''start_population denotes how many individuals/chromosomes are there
  in the initial population n = 8'''
start_population = 8 

'''if you want you can set mutation_threshold to a higher value,
   to increase the chances of mutation'''
mutation_threshold = 0.3

'''creating the population with random integers between 0 to 7 inclusive
   for n = 8 queen problem'''
population = np.random.randint(0, n, (start_population, n))
'''calling the GA function'''
i = 0
max_fitness_found = False
while i<100 and max_fitness_found==False:
  visited = np.zeros(n)
  population = GA(population, n, mutation_threshold)
  i+=1  


Chromosome:[0 5 2 0 3 6 5 2]  Fitness:22
Chromosome:[6 1 4 1 3 2 6 4]  Fitness:21
Chromosome:[1 5 1 2 7 4 4 4]  Fitness:20
Chromosome:[1 4 5 5 0 3 4 1]  Fitness:20
Chromosome:[4 1 5 5 0 0 3 7]  Fitness:24
Chromosome:[2 7 2 6 3 1 5 1]  Fitness:24
Chromosome:[3 2 1 3 2 1 5 7]  Fitness:17
Chromosome:[2 7 7 4 6 3 3 1]  Fitness:21

Chromosome:[0 7 2 6 3 1 5 1]  Fitness:24
Chromosome:[4 1 5 5 0 0 3 1]  Fitness:23
Chromosome:[3 4 7 4 6 3 3 1]  Fitness:21
Chromosome:[2 7 1 3 2 4 5 7]  Fitness:21
Chromosome:[4 1 5 5 7 0 3 7]  Fitness:24
Chromosome:[1 4 5 3 0 3 4 1]  Fitness:22
Chromosome:[1 4 5 5 0 6 4 1]  Fitness:22
Chromosome:[1 5 1 2 7 4 4 4]  Fitness:20

Chromosome:[1 5 5 5 0 6 4 1]  Fitness:22
Chromosome:[1 5 7 2 7 4 4 4]  Fitness:21
Chromosome:[3 4 7 4 2 4 5 7]  Fitness:21
Chromosome:[2 7 1 7 6 3 3 1]  Fitness:20
Chromosome:[4 1 5 3 0 0 3 1]  Fitness:23
Chromosome:[3 4 7 4 6 7 3 1]  Fitness:22
Chromosome:[1 5 5 3 0 3 4 1]  Fitness:22
Chromosome:[1 7 5 5 0 6 4 1]  Fitness:22

Chromosome:[1